**MODEL LOADING**

In [ ]:
!pip install transformers accelerate bitsandbytes torch
!pip install youtube-transcript-api chromadb

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from youtube_transcript_api import YouTubeTranscriptApi
from chromadb import Client

model_name = "HuggingFaceH4/zephyr-7b-beta"
youtube_video_id = "J5_-l7WIO_w"
print(f"Loading model: {model_name}")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    load_in_8bit=True
)
print("Model loaded.")

***Chain Building***

In [8]:
import warnings
warnings.filterwarnings("ignore", message="MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization")

youtube_video_id = "ukzFI9rgwfU"


# --- Fetch YouTube Transcript ---
print(f"Fetching transcript for video ID: {youtube_video_id}")
try:
    transcript_list = YouTubeTranscriptApi.get_transcript(youtube_video_id)
    transcript_text = " ".join([item['text'] for item in transcript_list])
    print("Transcript fetched successfully.")
except Exception as e:
    print(f"Error fetching transcript: {e}")
    transcript_text = ""

client = Client()
collection_name = f"youtube_transcript_{youtube_video_id}"
try:
    collection = client.get_collection(collection_name)
    print(f"Using existing collection: {collection_name}")
except:
    collection = client.create_collection(collection_name)
    print(f"Created new collection: {collection_name}")

if transcript_text:
    sentences = transcript_text.split('.')
    ids = [f"chunk_{i}" for i in range(len(sentences))]
    documents = [sentence.strip() for sentence in sentences if sentence.strip()]

    if documents:
        print(f"Adding {len(documents)} document chunks to ChromaDB.")
        collection.add(
            documents=documents,
            ids=ids
        )
        print("Documents added to ChromaDB.")
    else:
        print("No valid document chunks to add.")
else:
    print("No transcript available to store in ChromaDB.")

def query_transcript(query_text, n_results=5):
    """
    Queries the ChromaDB collection for relevant chunks and uses the LLM
    to generate a response based on the query and retrieved context.
    """
    print(f"\nQuerying for: {query_text}")
    results = collection.query(
        query_texts=[query_text],
        n_results=n_results
    )

    retrieved_chunks = results['documents'][0]
    print(f"Retrieved {len(retrieved_chunks)} relevant chunks.")

    if not retrieved_chunks:
        return "Could not find relevant information in the transcript."
    context = "\n".join(retrieved_chunks)
    prompt = f"""Using the following context, answer the question.
    If the answer is not in the context, say "I could not find the answer in the transcript."
    Context:
    {context}
    Question: {query_text}

    Answer:
    """

    print("Generating response with LLM...")
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            top_k=50,
            top_p=0.95
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer_start_index = response.find("Answer:")
    if answer_start_index != -1:
        generated_answer = response[answer_start_index + len("Answer:"):].strip()
    else:
        generated_answer = response.strip()
    return generated_answer

print("Training Complete")

Fetching transcript for video ID: ukzFI9rgwfU
Error fetching transcript: no element found: line 1, column 0
Using existing collection: youtube_transcript_ukzFI9rgwfU
No transcript available to store in ChromaDB.
Training Complete


In [9]:
user_query = "what is machine learning?"
answer = query_transcript(user_query)
print("\nLLM Answer:")
print(answer)


Querying for: what is machine learning?
Retrieved 1 relevant chunks.
Generating response with LLM...

LLM Answer:
Machine learning is a process by which machines can learn from past data and experiences, make predictions or decisions, and understand and reason based on that learning. It goes beyond just learning, as it involves understanding the relationships between different variables and the outcomes of those variables. Machine learning involves different approaches, such as supervised learning, unsupervised learning, and reinforcement learning, depending on whether the data is labeled or unlabeled and whether feedback or rewards are provided. The availability of large amounts of data, advances in computer memory and processing capabilities, and applications in various industries, such as healthcare, finance, and transportation, make machine learning a rapidly growing field in technology.

    Context:
    We know humans learn from their past experiences and machines follow instruc